<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/metodos_iterativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1></h1>
<h1><center>Cálculo Numérico<br> 2025/2</center></h1>
<center>Prof. Fernando Deeke Sasse - CCT, UDESC</center>

<h2><center>Métodos Iterativos e Diretos para Sistemas Lineares Esparsos</center></h2>

## 1. Introdução

Já vimos os chamados métodos diretos para sistemas lineares, baseados no processo de eliminação gaussiana. Estes métodos estão implementados de maneira eficiente o otimizada desde os ano 60 em Fortran. Os mesmos algoritmos e bibliotecas numéricas estão disponíveis em quase todos os sistemas computacionais. No entanto, quando temos sistemas muito grandes, que ocorrem frequentemente em problemas práticos,  os métodos iterativos são mais eficientes e este será o tema desta seção.

Para que tenhamos noção do que significa "grande" nos dias de hoje (2020) apresentaremos algumas classificações advindas da prática baseadas na ordem $n$ dos sistemas:
1. $n<10^3$: minúsculo.
2. $n<4 \times 10^3$: pequeno.
3. $n>4 \times 10^3$: grande.
Mesmo sistemas com até um milhão de componentes têm se tornado ultimamente mais comuns. Neste caso, os métodos diretos não podem ser mais ser aplicados.

Nestes casos, para o caso de sistemas grandes, devemos recorrer aos chamados métodos iterativos. Frequentemente sistemas grandes são esparsos: ou seja, somente algumas poucas componentes da matriz de coeficientes são não nulas. Este é o caso de

1. Discretização de malhas em equações diferenciais parciais.
2. Matrizes adjacentes em grandes grafos e redes.

Portanto, métodos iterativos eficientes devem fazer referência somente a elementos não nulos.

## 2. Ideia geral do método iterativo

Suponhamos que temos um sistema de equações da forma $AX=b$, sendo $A$ uma matriz não singular $n \times n$. A ideia dos métodos iterativos consistem em reescrever a equação matricial na forma

$$
X=BX+c\,,
$$

sendo $B$ uma matriz $n \times n$, chamada *matriz de iteração*  e $c$ uma matriz coluna (ou vetor). Tal rearranjo pode ser feito de inúmeras formas, cada uma definindo um diferente método iterativo. O processo iterativo é então definido da forma:

$$
X^{(k+1)}=BX^{(k)}+c\,,
$$
a partir de uma aproximação inicial $X^{(0)}$. Por exemplo, se escrevermos $A$ na forma $A=(A-I)+I$, sendo $I$ a matriz identidade, o sistema de equações poderá ser reescrito como

$$
(A-I)X+X=b\,,
$$

de modo que

$$
X=-(A-I)X+b\,.
$$

Portanto, a matriz de iteração deste método simples é $B=-(A-I)$ e $g=b$. O correspondente método iterativo é dado por

$$
X^{(k+1)}=-(A-I)X^{(k)}+b\,.
$$

Como exemplo, definiremos uma função que realiza tais iterações:

In [ ]:
import numpy as np

In [ ]:
def it_simples(A,b, N):
    n = len(A[0])
    x = np.zeros(n)
    ID = np.identity(n)
    B = -(A - ID)
    for i in range(N):
        x = b+np.dot(B,x)
    return x

Testaremos o método no sistema $AX=b$ da forma:

In [ ]:
A = np.array([[0.13,0.03,0.01],[0.05,0.025,0.],[0.,0.06,0.13]])
b = np.array([0.01,0.03,0.01])

Apliquemos a função, realizando 5000 iterações:

In [ ]:
Xs=it_simples(A,b,500)
Xs

Aparentemente o processo convergiu para um valor finito. Calculemos o resíduo do resultado:

In [ ]:
R = np.dot(A,Xs)-b
R

Isso mostra que o resultado foi alcançado. Este método, no entanto, funciona somente para alguns sistemas lineares. Por exemplo, seja o sistema $AX=b$ da forma

In [ ]:
A = 100*np.array([[0.13,0.03,0.01],[0.05,0.025,0.],[0.,0.06,0.13]])
b = 100*np.array([0.01,0.03,0.01])

In [ ]:
A

In [ ]:
b

Realizando 10 iterações obtemos:

In [ ]:
Xs=it_simples(A,b,10)
Xs

o que mostra que processo está divergindo para vetores com componentes de magnitudes muito grandes.

## 3. Método de Jacobi

Descreveremos agora um método iterativo mais eficiente que o anterior. Para o sistema $AX=b$ reescrevemos $A$ na forma

$$
A = (A-D)+D\,,
$$

sendo $D$ a matrix que contém somente a parte da diagonal principal de $A$, ou seja,

$$
D = diag(A)\,.
$$

Portanto, o sistema pode ser reescrito como
$$
[(A-D)+D]X=b
$$
ou
$$
DX = b-(A-D)X\,.
$$
O processo iterativo de Jacobi é então dado por
$$
X^{(k+1)}=-D^{-1}(A-D)X^{(k)}+D^{-1}b\,,
$$
de modo que a matriz de iteração de Jacobi é dada por
$$
B_J = -D^{-1}(A-D)\,.
$$

Escreveremos uma função que realiza este processo iterativo:

In [ ]:
def jacobi(A,b, N):
    x = np.zeros(len(A[0]))
    D = np.diag(A)
    H = -(A - np.diagflat(D))
    for i in range(N):
        x = (b+np.dot(H,x))/D
    return x

Usemos o sistema da seção anterior:

In [ ]:
A = np.array([[0.13,0.03,0.01],[0.05,0.025,0.],[0.,0.06,0.13]])
b = np.array([0.01,0.03,0.01])

Realizemos 20 iterações:

In [ ]:
XJ = jacobi(A,b,20)
XJ

Calculemos o resíduo:

In [ ]:
R = np.dot(A,Xs)-b
R

Aqui o processo divergiu. Tentemos uma matriz diagonalmente dominante:

In [ ]:
A = np.array([[13.,2.3,4.2],[4.3,14.3,4.3],[3.3,4.1,10.4]])
b = np.array([1.2,2.1,1.5])

In [ ]:
XJ = jacobi(A,b,50)
XJ

In [ ]:
R = np.dot(A,XJ)-b
R

Portanto, com 50 iterações alcançamos um resultado com considerável acurácia. Veremos a seguir critérios de convergência que podem nos ajudar a garantir a convergência de um dado método aplicado a um dado sistema.

## 4. Convergência

Uma condição necessária e suficiente para a convergência de um processo iterativo da forma

$$
X^{(k+1)}=BX^{(k)}+g\,,
$$

é que todos os autovalores $\lambda_{i}$ de $B$ sejam tais que $|\lambda_i|<1$. No entanto, tal condição é difícil de ser verificada para sistemas grandes. Um critério que é somente suficiente, mas muito mais prático, é que a norma de $B$ seja menor que a unidade, ou seja, $||B||_{\infty}<1$. Consideremos alguns exemplos. Consideremos a matriz de coeficientes:  

In [ ]:
A = np.array([[13.,2.3,4.2],[4.3,14.3,4.3],[3.3,4.1,10.4]])

A correspondente matriz de iteração de Jabobi $B_J = -D^{-1}(A-D)$ é

In [ ]:
D = np.diag(A)
BJ = -(A - np.diagflat(D))/D
BJ

Calculemos a norma infinito desta matriz:

In [ ]:
from numpy import linalg as LA

In [ ]:
LA.norm(BJ,np.inf)

de modo que a convergência do processo de Jacobi para qualquer sistema da forma $AX=b$ é garantida. Definamos uma função que tem como entrada a matriz de coeficientes e saída a norma da matriz de iteração de Jacobi:  

In [ ]:
def jacobi_norm(A):
    D = np.diag(A)
    BJ = -(A - np.diagflat(D))/D
    norma = LA.norm(BJ,np.inf)
    return norma

Testemos a função:

In [ ]:
jacobi_norm(A)

Para ilustrar o fato de que a condição sobre a norma da matriz de iteração é somente uma condição suficiente mas não necessária, consideremos agora um sistema levemente diferente do anterior:

In [ ]:
A1 = np.array([[13.,2.3,14.2],[4.3,14.3,4.3],[3.3,4.1,10.4]])
b1 = np.array([1.2,2.1,1.5])

A norma da respectiva matriz de iteração é dada por

In [ ]:
jacobi_norm(A1)

de modo que a convergência não é garantida. Tentaremos o processo de Jacobi mesmo assim:

In [ ]:
XJ = jacobi(A1,b1,100)
XJ

Aparentemente o processo convergiu. Examinemos o resíduo:

In [ ]:
R = np.dot(A1,XJ)-b1
R

Isso mostra que a solução foi obtida.

## 5. Critério de Parada

Nas funções que definimos acima o número de iterações é fornecido na entrada. No entanto é interessante usar um critério de parada para o processo iterativo. O critério mais eficiente e compucionalmente barato consiste em estabelecer uma certa tolerância $\epsilon$ e interromper o processo quando

$$
||X^{(k+1)}-X^{(k)}||_{\infty}<\epsilon
$$

Acrescentameremos a tolerância $\epsilon$ à função *jacobi* definida anteriormente. Devemos estabelecer um número máximo de iterações, caso o processo seja divergente.

In [ ]:
def jacobi2(A,b, N,epsilon):
    x = np.zeros(len(A[0]))
    D = np.diag(A)
    H = -(A - np.diagflat(D))
    for i in range(N):
        xn = (b+np.dot(H,x))/D
        norma = LA.norm(xn-x)
        if norma < epsilon:
            break
        x=xn
    return x

Testemos a nova função:

In [ ]:
A1 = np.array([[13.,2.3,14.2],[4.3,14.3,4.3],[3.3,4.1,10.4]])
b1 = np.array([1.2,2.1,1.5])

Estabeleçamos uma tolerância $\epsilon = 10^{-5}$ e um número máximo de 500 iterações:

In [ ]:
XJ2 = jacobi2(A1,b1, 500,1e-5)
XJ2

Exeminemos o resíduo:

In [ ]:
R2 = np.dot(A1,XJ2)-b1
R2

de modo que a solução foi obtida, compatível com a tolerância estabelecida.

A seguinte função fornece na saída o número de iterações realizadas:

In [ ]:
def jacobi3(A,b, N,epsilon):
    x = np.zeros(len(A[0]))
    D = np.diag(A)
    H = -(A - np.diagflat(D))
    iter = 0
    for i in range(N):
        xn = (b+np.dot(H,x))/D
        norma = LA.norm(xn-x)
        if norma < epsilon:
            break
        x=xn
        iter = iter + 1
    return (x,iter)

Testemos o procedimento, usando os mesmos argumentos de exemplo anterior:

In [ ]:
A1 = np.array([[13.,2.3,14.2],[4.3,14.3,4.3],[3.3,4.1,10.4]])
b1 = np.array([1.2,2.1,1.5])

In [ ]:
sol = jacobi3(A1,b1, 500,1e-5)
sol

Houve, portanto, 56 iterações. O valor do vetor solução é a primeira componente deste *array*:

In [ ]:
XJ3 = sol[0]
XJ3

## 6. Método de Gauss-Seidel

O método de Gauss-Seidel é, na maioria dos casos, mais eficiente do que o método de Jacobi.  Ele é implementado a partir da seguinte decomposição da matriz de coeficientes $A$:
$$
A = L+D+U\,,
$$
sendo $L$ a parte abaixo da diagonal  de $A$, $D$ é a parte da diagonal de $A$ e $U$ é a parte acima da diagonal de $A$.
Portanto, podemos escrever

$$
(L+D+U)X = b\,,
$$

ou

$$
(L+D)X = -UX+b\,.
$$

Embora a correspondente forma matricial do processo iterativo seja formalmente

$$
X^{(k+1)} =(L+D)^{-1}(-UX^{(k)}+b)\,,
$$

não é esta a forma implementada na prática. Aqui devemos olhar novamente a fórmula $(L+D)X^{(k+1)} = -UX^{(k)}+b$ e examinar como cada componente é calculada. Vejamos o caso particular de um sistema $3 \times 3$:
$$
A = \begin{bmatrix}
a_{11} & 0  & 0 \\\
a_{21} & a_{22}  & 0  \\\
a_{31} & a_{32}  & a_{33}
\end{bmatrix}
\begin{bmatrix}
x_1^{(k+1)}  \\
x_2^{(k+1)}  \\
x_3^{(k+1)} \\
\end{bmatrix}\, =
-\begin{bmatrix}
0 & a_{12}  & a_{13} \\\
0 & 0  & a_{23} \\\
0 & 0  & 0
\end{bmatrix}\begin{bmatrix}
x_1^{(k)}  \\
x_2^{(k)}  \\
x_3^{(k)} \\
\end{bmatrix}\,+\, \begin{bmatrix}
b_1 \\
b_2  \\
b_3 \\
\end{bmatrix}\,
$$

Resolvendo para a primeira componente obtemos:

$$
x_1^{(k+1)} =\frac{1}{a_{11}}\left(b1-a_{12}x_2^{(k)}-a_{13}x_3^{(k)}\right)\,,
$$

O valor de $x_1$ agora será atualizado, de modo que a próxima componente é dada por

$$
x_2^{(k+1)} =\frac{1}{a_{22}}\left(b2-a_{21}x_1^{(k+1)}-a_{23}x_3^{(k)}\right)\,.
$$

Similarmente,

$$
x_3^{(k+1)} =\frac{1}{a_{33}}\left(b2-a_{31}x_1^{(k+1)}-a_{32}x_2^{(k+1)}\right)\,.
$$

Notemos que tal procedimento é similar ao do método de Jacobi, exceto pelo fato de que os valores das sucessivas componentes não é atualizado a cada passo. Uma implementação computacional deve, portanto, calcular cada componente de $X$. O preço pago pela perda da vetorização do cálculo é compensado pela convergência mais rápida, como veremos em seguida. Podemos implmentar este cálculo em Python do por meio da seguinte função que tem como entrada as matrizes $A$ e $b$ do sistemas $AX=b$ e o número $N$ de iterações. Como fizemos antes, o valor inicial do processo iterativo é já definido dentro da função como sendo o vetor nulo.

In [ ]:
import numpy as np
from numpy import linalg as LA

In [ ]:
def gauss_seidel1(A,b, N):
    n = len(A[0])
    x = np.zeros(n)
    D = np.diag(A)
    H = -(A - np.diagflat(D))
    for i in range(N):
        for j in range(n):
            x[j] = (b[j]+np.dot(H[j,:],x))/D[j]
    return x

Testemos a função num sistema $3 \times 3$, fazendo 10 iterações:

In [ ]:
A1 = np.array([[13.,2.3,14.2],[3.3,14.3,4.3],[3.3,4.1,10.4]])
b1 = np.array([1.2,2.1,1.5])

In [ ]:
Xgs = gauss_seidel1(A1,b1, 15)
Xgs

Calculemos o resíduo:

In [ ]:
R1gs = np.dot(A1,Xgs)-b1
R1gs

Comparemos este resultado com aquele obtido pelo método de Jacobi:

In [ ]:
def jacobi1(A,b, N):
    x = np.zeros(len(A[0]))
    D = np.diag(A)
    H = -(A - np.diagflat(D))
    for i in range(N):
        x = (b+np.dot(H,x))/D
    return x

In [ ]:
XJ1 = jacobi1(A1,b1, 50)
XJ1

In [ ]:
R1j = np.dot(A1,XJ1)-b1
R1j

Portanto, em 15 iterações o método de Gauss-Seidel forneceu um resultado com uma acurácia muito superior àquela do método de Jacobi com 50 iterações. No exemplo seguinte, encontramos outra situação frequentemente observada: o método de Gauss-Seidel converge enquanto que o de Jacobi diverge. Usemos uma matriz de coeficientes que não é diagonalmente dominante:

In [ ]:
A2 = np.array([[13.,2.3,14.2],[24.3,14.3,4.3],[3.3,4.1,10.4]])
b2 = np.array([1.2,2.1,1.5])

In [ ]:
Xgs2 = gauss_seidel1(A2,b2, 15)
Xgs2

In [ ]:
R2gs = np.dot(A2,Xgs2)-b2
R2gs

In [ ]:
XJ2 = jacobi1(A2,b2, 50)
XJ2

Embora seja um evento raro, é possível que para alguns sistemas lineares o método de Jacobi convirja e o de Gauss-Seidel divirja. Vejamos um exemplo:

In [ ]:
A3 = np.array([[1.,2.,-2.],[1.,1.,1.],[2.,2.,1.]])
b3 = np.array([3.,2.,1.])

In [ ]:
XJ3 = jacobi1(A3,b3, 50)
XJ3

In [ ]:
R1j = np.dot(A1,XJ1)-b1
R1j

In [ ]:
Xgs3 = gauss_seidel1(A3,b3, 50)
Xgs3

Reiteramos que tal situação é uma exceção. Normalmente o método de Gauss-Seidel converge quando o de Jacobi não converge e quando ambos convergem, Gauss-Seidel é muito mais rápido.

Como no método de Gauss-Seidel a variável $x$ tem suas componentes atualizadas sequencialmente, não é possível implementar um critério de parada do mesmo jeito que fizemos para o método de Jacobi. A seguinte implementação (menos elegante) do método de Gauss-Seidel tem como entrada a tolerância como critério de parada, além do número máximo de iterações:

In [ ]:
def gauss_seidel2(a,b, N,epsilon):
    n = len(a[0])
    x = np.zeros(n)
    xdiff = np.empty(n, float)
    for iteration in range(N):
        for i in range(n):
            s = 0
            for j in range(n):
                if j != i:
                    s += a[i, j]*x[j]
            xnew = -1/a[i,i] * (s - b[i])
            xdiff[i] = abs(xnew - x[i])
            x[i] = xnew
        if(xdiff < epsilon).all():
             break
    return (x, iteration+1)

Testemos a função no sistema usado acima:

In [ ]:
A1 = np.array([[13.,2.3,14.2],[3.3,14.3,4.3],[3.3,4.1,10.4]])
b1 = np.array([1.2,2.1,1.5])

In [ ]:
(Xgs2,it) = gauss_seidel2(A1,b1, 30,1e-8)
print(Xgs2,it)

de modo que 18 iterações foram necessárias. Computemos o resíduo:

In [ ]:
R2gs = np.dot(A1,Xgs2)-b1
R2gs

Exercício: Determine quantas iterações o método do Jacobi requer para alcançar esta tolerância.

## 7. Critérios de Convergência para a Matriz de Coeficientes para os Métodos de Jacobi e Gauss-Seidel

### 7.1 Critérios das linhas e colunas

É fácil mostrar [1] que, no caso do processo de Jacobi, a condição $\parallel B \parallel{}_{\infty} <1$ é equivalente ao chamado *critério das linhas*:

$$
\mbox{max}_{1\leq i \leq n} \left(\sum_{j=1,j\neq i}\frac{\left|a_{ij}\right|}{\left|a_{ii}\right|}\right)<1
$$

e que $\parallel B \parallel_{1} <1$ corresponde ao chamado *critério das colunas*:

$$
\mbox{max}_{1\leq j \leq n} \left(\sum_{i=1,i\neq j}\frac{\left|a_{ij}\right|}{\left|a_{jj}\right|}\right)<1
$$

Implementemos estas condições suficientes para convergência:

In [ ]:
import numpy as np
from numpy import linalg as LA

In [ ]:
def criterio_linha(A):
    n = len(A[0])
    a = np.zeros(n)
    D = np.diag(A)
    DM = np.diagflat(D)
    M = A - DM
    for i in range(n):
        a[i] =LA.norm(M[i,:]/D[i],1)
    alpha = LA.norm(a,np.inf)
    return alpha

Na função acima a norma 1 é usada para somar as componentes de cada linha $i$ da matriz $M$. Testemos a função.  

In [ ]:
A3 = np.array([[5.,1.,-2.],[1.,4.,1.],[2.,2.,7.]])

In [ ]:
len(A3)

In [ ]:
alpha = criterio_linha(A3)
alpha

Para o critério de colunas basta tomar as colunas de $M=A-diag(A)$:

In [ ]:
def criterio_coluna(A):
    n = len(A)
    a = np.zeros(n)
    D = np.diag(A)
    DM = np.diagflat(D)
    M = A - DM
    for i in range(n):
        a[i] =LA.norm(M[:,i]/D[i],1)
    alpha = LA.norm(a,np.inf)
    return alpha

In [ ]:
beta = criterio_coluna(A3)
beta

É possível mostrar que ambos os critérios garantem também a convergência do método de Gauss-Seidel. Neste caso, uma condição suficiente mais fraca do que a das linhas (e portanto mais útil) é a de Sassenfeld (veja [1] ou [2]).

In [ ]:
def criterio_sassenfeld(A):
    coef = []
    n = len(A)
    for i in range(n):
        bb=0
        for j in range(n):
            if (i!=j and i==0) or i<j:
                bb+=A[i][j]
            elif i!=j and i!=0:
                 bb+=A[i][j]*coef[j]
        bb/=A[i,i]
        coef.append(bb)
    beta = max(coef)
    return beta

In [ ]:
A4 = np.array([[13,3,1],[5,2.5,0.],[10,6,13]])

In [ ]:
criterio_sassenfeld(A4)

## 8. Métodos Diretos para Sistemas Esparsos

Embora o tema desta lição sejam métodos iterativos, é interessante examinar métodos diretos otimizados para sistemas esparsos. Em particular, usaremos o SuperLU 4.0, incluído do SciPy.

Aqui estão as especificações no manual:



    default solver: SuperLU 4.0
            included in SciPy
            real and complex systems
            both single and double precision

    optional: umfpack
            real and complex systems
            double precision only
            recommended for performance
            wrappers now live in scikits.umfpack
            check-out the new scikits.suitesparse by Nathaniel Smith



In [ ]:
import numpy as np
import scipy.sparse as sps
from scipy.sparse.linalg import dsolve

Para maior efeciência do processo, a matriz de coeficientes $A$ pode ser definida no formato csc (Compressed Sparse Column matrix):

In [ ]:
A = csc_matrix([[1., 2., -41., 4., 25.], [72., -51., 8., 9., 1.3], [100.,3.,4.,5.,2.], [2.,3,.4,.43,2],[2,5,2,3,10]], dtype=float)
A

Como esta é uma matriz pequena, podemos visualisá-la:

In [ ]:
A.todense()

In [ ]:
b = np.array([1, 2, 3, 4, 5], dtype=np.float64)

In [ ]:
X= dsolve.spsolve(A, b, use_umfpack=False)
X

In [ ]:
R = np.linalg.norm(A * X - b)
R

Façamos um teste com uma matriz esparsa grande.

In [1]:
import numpy as np
from scipy.sparse import coo_matrix

In [9]:
n = 10
k = 3 * n
rows = np.random.randint(0, n, size=k)
cols = np.random.randint(0, n, size=k)
data = np.random.rand(k)
A_coo = coo_matrix((data, (rows, cols)), shape=(n, n))
A = A_coo.tocsr()  # converte para CSR para operações rápidas.
b = np.ones((n, 1))

In [10]:
A

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 25 stored elements and shape (10, 10)>

In [11]:
A.todense()

matrix([[1.48342001, 0.        , 0.        , 0.0840641 , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.61165537, 0.        , 0.79379716,
         0.75920328, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.81648423, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.69671567, 0.        ,
         0.        , 0.        , 0.16117961, 0.13310102, 0.        ],
        [0.        , 0.75872428, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.39948307,
         0.        , 1.10001498, 0.59696871, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.24547076, 0.        ,
         0.88514671, 0.        , 0.92005924, 0.75909026, 0.        ],
        [0.        , 0.4854

In [12]:
print(A.shape, A.nnz)

(10, 10) 25


Resolvamos o sistema:

In [13]:
from scipy.sparse.linalg import spsolve

In [15]:
x = spsolve(A, b)
x

array([ 0.5653584 ,  1.31800184,  1.2247634 ,  1.91920264, -2.33451127,
        2.77132007,  2.66400889, -1.6715314 , -0.50879994,  1.1438532 ])

## 9. Métodos Iterativos para Sistemas Esparsos

Os seguintes métodos iterativos são disponíveis em Scipy [3]


bicg(A, b[, x0, tol, maxiter, M, callback, atol])
Use BIConjugate Gradient iteration to solve Ax = b.

bicgstab(A, b[, x0, tol, maxiter, M, …])
Use BIConjugate Gradient STABilized iteration to solve Ax = b.

cg(A, b[, x0, tol, maxiter, M, callback, atol])
Use Conjugate Gradient iteration to solve Ax = b.

cgs(A, b[, x0, tol, maxiter, M, callback, atol])
Use Conjugate Gradient Squared iteration to solve Ax = b.

gmres(A, b[, x0, tol, restart, maxiter, M, …])
Use Generalized Minimal RESidual iteration to solve Ax = b.

lgmres(A, b[, x0, tol, maxiter, M, …])
Solve a matrix equation using the LGMRES algorithm.

minres(A, b[, x0, shift, tol, maxiter, M, …])
Use MINimum RESidual iteration to solve Ax=b

qmr(A, b[, x0, tol, maxiter, M1, M2, …])
Use Quasi-Minimal Residual iteration to solve Ax = b.

gcrotmk(A, b[, x0, tol, maxiter, M, …])
Solve a matrix equation using flexible GCROT(m,k) algorithm.

Ilustraremos somente o uso da função bicg.

In [ ]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import bicg
from numpy import linalg as LA
import numpy as np

Como exemplo usemos uma matriz não diagonalmente dominante:

In [ ]:
A = csc_matrix([[3, 2, 0], [1, -1, 2], [0, 5, 1]])
b = np.array([2, 4, -1], dtype=float)
print(A)

In [ ]:
A.todense()

In [ ]:
x, exitCode = bicg(A, b)

In [ ]:
print(exitCode)  # 0 indicates successful convergence

In [ ]:
x

Podemos verificar que esta é realmente a solução, calculando a norma infinito do resíduo:

In [ ]:
import numpy.linalg as la

In [ ]:
R = la.norm(A * x - b, np.inf)
R

## 10. Exercícios

1. Este problema deve ser resolvido passo a passo, usando Python somente como calculadora. Considere o sistema linear  $AX = B$ com

 $$
 A= \left[ \begin {array}{cccc}  0.086302& 0.11812& 0.19083& 0.082780
\\  0.14301& 0.94223& 0.16756& 0.072127
\\  0.090268& 0.056300& 0.92133& 0.066085
\\  0.0095109& 0.071374& 0.13895& 0.93211
\end {array} \right] \,,
$$

$$
B = [1,1,1,1]^T\,.
$$

(i)  Verifique se a condição necessária e suficiente para convergência é satisfeita para a matriz de iteração de Jacobi.

(ii) Verifique se a condição suficiente, envolvendo somente norma matricial,  é satisfeita para a matriz de iteração de Jacobi.

(iii) Use o critério de linhas para analisar a convergência do método de Jacobi.

(iv) Utilize o critério de Sassenfeld para verificar se a convergência do método de Gauss-Seidel é garantida.

(v) Aplique o método de Gauss-Seidel e determine se  o processo converge. Em caso positivo, determine a solução.

(vi) Em caso de convergência, determine o número de iterações de Gauss-Seidel necessárias para que o erro entre a norma infinito de aproximações sucessivas seja menor que $10^{-10}$

(vii) Repita o cálculo anterior para para o método de Jacobi.

2. Resolva novamente os itens problema 1, agora usando funções do Python definidas anteriormente, com

$$
A =\left[ \begin {array}{ccccc} 1&2&1&1&0\\ 3&7& 0.3&
 1.1& 2.1\\ 0& 1.2& 8.3& 2.2& 5.3
\\  2.5& 2.7& 0.7& 9.8& 2.1\\  1.1
& 2.1& 0.7& 1.4& 8.2\end {array} \right]\,,
$$

$$
B = [1,1,1,1,1]^T\,.
$$

Acrescentando mais um item, resolva o problema usando a função bigcg do Scipy. Calcule o tempo de CPU e compare com aquele obtido por meio de um método direto (do Numpy ou Scipy).


3. Considere este problema resolvido [4] (https://bit.ly/2NDLqRG). (i) Verifique se ele pode ser resolvido por métodos iterativos. Em caso positivo, resolva-o no Python. (ii) Resolva o problema novamente utilizando fatoração LU (comando do Scipy). Note que problemas práticos podem envolver estruturas muito grandes e, portanto, muito mais equações.


4. Resolva estes problemas [4] (https://bit.ly/2O1FHVu) com Python, utilizando qualquer método.

## Referências

1. Neide Bertoldi Franco. Cálculo Numérico, Pearson, 2007. https://bit.ly/2E9mY5I
2. Yara de Souza Tadano. Métodos Iterativos, Notas de Aula 13: https://bit.ly/3gahu7Q
3. Scipy Sparse linear algebra, Reference Guide: https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html
4. Steven C. Chapra, Raymond P. Canale. Métodos Numéricos para Ingenieros, 7 ed.,  The McGraw-Hill Education, 2015.